In [28]:
from pymilvus import connections, Collection, utility
from pymilvus import utility
import google.generativeai as genai
from local_settings import API_KEY,uri,token
from pymilvus import MilvusClient
from helper_func import create_prompt, process_data

connections.connect(uri=uri,token=token)
milvus_client = MilvusClient(uri=uri,token=token)

collection_name = "my_rag_collection2"

In [29]:

collection = Collection(collection_name)
collection.num_entities 

21326

In [30]:
utility.list_collections()

['my_rag_collection2']

In [31]:
res = collection.query(expr="", output_fields=["count(*)"])
print(res)

[{'count(*)': 21326}]


In [32]:
embedding_model = 'models/text-embedding-004'
genai.configure(api_key=API_KEY)
def embed_fn(title, text):
  return genai.embed_content(model=embedding_model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

In [33]:
question = "Can Home Insurance Companies Drop You? "

search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        embed_fn(question,question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=7,  # Return top 3 results
    search_params={"metric_type": "COSINE",  "params": {"level": 2}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

In [34]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
retrieved_lines_with_distances

[(['Can My Home Insurance Company Drop Me? ',
   "Yes it can but for very specific reasons that are filed and approved by your state 's Insurance Department . Just like a company offering new coverage provided that you and your property meet filed underwriting guidelines , if while your policy is in force , you or your property no longer meet those guidelines for reasons such as prior losses , foreclosure , property abandonment , failure to pay your premium or other approved reasons , your company can either cancel mid term or non renew . "],
  0.8964682817459106),
 (['Can A Home Insurance Company Drop You? ',
   'An insurance carrier may decide to cancel or non-renew a client if the client no longer meets the carriers standard for providing coverage . A client could also be dropped or non-renewed by a carrier if the coverage provided was approved under false information or fraudulant acts . If you receive a letter of cancellation or non-renewal from your current carrier you should con

In [35]:

querry_data= process_data(retrieved_lines_with_distances)
print(querry_data)

Q: Can My Home Insurance Company Drop Me?

A: Yes it can but for very specific reasons that are filed and approved by your state 's Insurance Department . Just like a company offering new coverage provided that you and your property meet filed underwriting guidelines , if while your policy is in force , you or your property no longer meet those guidelines for reasons such as prior losses , foreclosure , property abandonment , failure to pay your premium or other approved reasons , your company can either cancel mid term or non renew .

Q: Can A Home Insurance Company Drop You?

A: An insurance carrier may decide to cancel or non-renew a client if the client no longer meets the carriers standard for providing coverage . A client could also be dropped or non-renewed by a carrier if the coverage provided was approved under false information or fraudulant acts . If you receive a letter of cancellation or non-renewal from your current carrier you should contact your local agent to either co

In [36]:
prompt=create_prompt(querry_data,question)
print(prompt)


Task: Your are a Insurance assistance, your job is based on a conversation of a costumer and a insurance expert, 
answer insurance-related questions accurately and provide detailed  explanation.
Instructions:

Read the context carefully.

Consider key points, concepts, and details within the document.

Generate questions that seek information or clarification about the document's content.

Ensure that the questions are grammatically correct and understandable.

Give the answer directly, do not repeat the question or add any unrelated text

If the user talk to you casually, you can rely back

If the question is blank, tell the user to type the question in.

If the question is not related to the context or there is no answer in the context, tell the users the question is not related to insurance or there is no answer in your database

Context:

Q: Can My Home Insurance Company Drop Me?

A: Yes it can but for very specific reasons that are filed and approved by your state 's Insurance De

In [37]:
LLM_model = genai.GenerativeModel('gemini-pro')
response = LLM_model.generate_content(prompt)

In [38]:
print(response.text.replace("JSON", "").replace("```",""))

Yes, they can
